## Node Class

In [18]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [53]:
class Node:
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, value=None):
        self.feature_index = feature_index  # Index of the feature to split on
        self.threshold = threshold            # Threshold value for the split
        self.left = left                      # Left child node
        self.right = right                    # Right child node
        self.value = value                    # Class label for leaf nodes

    def is_leaf(self):
        return self.value is not None  # Check if the node is a leaf

## Tree Class

In [70]:
class MyDecisionTreeClassifier():
    def __init__(self, max_depth=100, min_samples_split=2):
        ''' 
        Constructor for the MyDecisionTreeClassifier.
        Parameters:
        - max_depth: The maximum depth of the tree (to prevent overfitting)
        - min_samples_split: The minimum number of samples required to split a node
        '''
        np.random.seed(10)
        self.root = None  # Initialize the root of the tree
        self.min_samples_split = min_samples_split  # Minimum samples required to split
        self.max_depth = max_depth  # Maximum depth of the tree

    def is_finished(self, depth):
        ''' 
        Determine if the tree construction should stop based on certain conditions.
        Parameters:
        - depth: Current depth of the tree
        Returns:
        - True if any stopping condition is met, otherwise False
        '''
        if depth >= self.max_depth or self.n_class_labels == 1 or self.n_samples < self.min_samples_split:
            return True
        return False

    def build_tree(self, X, y, depth=0):
        ''' 
        Recursively build the decision tree.
        Parameters:
        - X: Features of the training dataset
        - y: Labels of the training dataset
        - depth: Current depth of the tree (default is 0)
        Returns:
        - A Node representing the root of the constructed subtree
        '''
        self.n_samples, self.n_features = X.shape  # Get the number of samples and features
        self.n_class_labels = len(np.unique(y))     # Get the number of unique class labels

        # Check stopping criteria
        if self.is_finished(depth):
            # Assign the most common label to the leaf node
            most_common_label = np.argmax(np.bincount(y))
            return Node(value=most_common_label)

        # Find the best feature and threshold to split on
        rnd_feats = np.random.choice(self.n_features, self.n_features, replace=False)
        best_feat, best_thresh = self.best_split(X, y, rnd_feats)

        # Split the dataset into left and right nodes
        left_idx, right_idx = self.create_split(X[:, best_feat], best_thresh)
        left_child = self.build_tree(X[left_idx, :], y[left_idx], depth + 1)
        right_child = self.build_tree(X[right_idx, :], y[right_idx], depth + 1)

        # Create and return a node for the current best split
        return Node(best_feat, best_thresh, left_child, right_child)

    def fit(self, X, y):
        ''' 
        Fit the model to the training data.
        Parameters:
        - X: Features of the training dataset
        - y: Labels of the training dataset
        '''
        # Encode the labels using LabelEncoder
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)
        # Build the decision tree using the encoded labels
        self.root = self.build_tree(X, y_encoded)

    def entropy(self, y):
        ''' 
        Calculate the entropy of the labels.        
        Parameters:
        - y: Labels of the dataset
        Returns:
        - The entropy value
        '''
        # Calculate the proportions of each class
        proportions = np.bincount(y) / len(y)
        # Compute entropy
        entropy = -np.sum([p * np.log2(p) for p in proportions if p > 0])
        return entropy

    def create_split(self, X, threshold):
        ''' 
        Create a split of the dataset based on the threshold.
        Parameters:
        - X: Feature values to split
        - threshold: Threshold value to use for splitting
        Returns:
        - Indices of the left and right splits
        '''
        left_idx = np.argwhere(X <= threshold).flatten()  # Indices for left split
        right_idx = np.argwhere(X > threshold).flatten()   # Indices for right split
        return left_idx, right_idx
        
    def information_gain(self, X, y, threshold):
        ''' 
        Calculate the information gain from a split.
        Parameters:
        - X: Feature values to split
        - y: Labels of the dataset
        - threshold: Threshold value for the split
        Returns:
        - Information gain from the split
        '''
        parent_loss = self.entropy(y)  # Calculate entropy before the split
        left_idx, right_idx = self.create_split(X, threshold)  # Create split

        n, n_left, n_right = len(y), len(left_idx), len(right_idx)
        if n_left == 0 or n_right == 0: 
            return 0  # No gain if one of the splits is empty

        # Calculate child loss based on the left and right splits
        child_loss = (n_left / n) * self.entropy(y[left_idx]) + (n_right / n) * self.entropy(y[right_idx])
        return parent_loss - child_loss  # Return the information gain

    def best_split(self, X, y, features):
        ''' 
        Find the best feature and threshold to split on.
        Parameters:
        - X: Features of the training dataset
        - y: Labels of the training dataset
        - features: Randomly selected features to consider for the split
        Returns:
        - Best feature index and threshold value
        '''
        best_split = {'score': -1, 'threshold': None, 'feature': None}

        for feature in features:
            X_feature = X[:, feature]  # Get the feature values for the current feature
            thresholds = np.unique(X_feature)  # Unique threshold values for the feature

            for thresh in thresholds:
                score = self.information_gain(X_feature, y, thresh)  # Calculate information gain
                if best_split['score'] < score:  # Update best split if score is higher
                    best_split['score'] = score
                    best_split['threshold'] = thresh
                    best_split['feature'] = feature
                
        return best_split['feature'], best_split['threshold']  # Return the best feature and threshold

    def traverse_tree(self, x, node):
        ''' 
        Traverse the tree to make a prediction for a given input sample.
        Parameters:
        - x: Input sample to predict
        - node: Current node in the tree
        Returns:
        - Predicted class label for the input sample
        '''
        if node.is_leaf():
            return node.value  # Return the label of the leaf node

        # Decide to traverse left or right based on the feature value
        if x[node.feature_index] <= node.threshold:  
            return self.traverse_tree(x, node.left)  # Traverse left
        return self.traverse_tree(x, node.right)  # Traverse right

    def predict(self, X):
        ''' 
        Predict class labels for a set of input samples.
        Parameters:
        - X: Features of the input samples
        Returns:
        - Array of predicted class labels
        '''
        predictions = [self.traverse_tree(x, self.root) for x in X]  # Make predictions for all samples
        return np.array(predictions)  # Return as numpy array

In [71]:
def my_accuracy_score(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

# Prediction on Iris dataset
### Using My Custom Model

In [72]:
data = pd.read_csv('Iris.csv')
data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [73]:
X = data.iloc[:, 1:-1].values
Y = data.iloc[:, -1].values
print(X.shape)
print(Y.shape)
print(X[0])

(150, 4)
(150,)
[5.1 3.5 1.4 0.2]


In [74]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [75]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(120, 4)
(120,)
(30, 4)
(30,)


In [79]:
classifier = MyDecisionTreeClassifier(max_depth=120)
classifier.fit(X_train, y_train)
my_pred = classifier.predict(X_test)

label_encoder = LabelEncoder()
y_test = label_encoder.fit_transform(y_test)
my_acc_iris = my_accuracy_score(y_test, my_pred)
print(f"My Accuracy (Iris dataset):       {my_acc_iris*100:.2f} %")

My Accuracy (Iris dataset):       96.67 %


### Using Sk-Learn

In [85]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.2, random_state=41)
iris_cls = DecisionTreeClassifier(max_depth=10)
iris_cls.fit(X_train, y_train)
pred = iris_cls.predict(X_test)

acc_iris = accuracy_score(y_test, pred)
print(f"Sk-Learn Accuracy (Iris dataset):       {acc_iris*100:.2f} %")

Sk-Learn Accuracy (Iris dataset):       90.00 %


In [86]:
print(f"My Accuracy (Iris dataset):             {my_acc_iris*100:.2f} %")
print(f"Sk-Learn Accuracy (Iris dataset):       {acc_iris*100:.2f} %")

My Accuracy (Iris dataset):             96.67 %
Sk-Learn Accuracy (Iris dataset):       90.00 %
